In [1]:
import numpy as np
from numpy import random as rnd
from matplotlib import pyplot as plt
import os,datetime,time,warnings,sys,itertools,math,platform

import pandas as pd

from plotly import express

import torch
import torch.nn as nn
import torch.nn.functional as F

from rdflib import Namespace,URIRef,Graph,Literal
from rdflib.namespace import RDF,NamespaceManager
from urllib.parse import quote

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
if platform.system()=='Windows':
    path_to_dq_excel = r'D:\\UserData\\z003mxpm\\gitdir\\GITREPO\\TF_Data\Dropbox\\PhD Prep\\SHS DQ\\DQ Flags in CDC.xlsx'
    ontology_path = r'D:\\UserData\\z003mxpm\\gitdir\\GITREPO\\TF_Data\Dropbox\\PhD Prep\\Protege-5.5.0 for OSX\\Ontologies\\Ontologies for Protege\\DQ_Metrics_Ontology.owl'
else:
    path_to_dq_excel = r'../../../../../../PhD Prep/SHS DQ/DQ Flags in CDC.xlsx'
    ontology_path = r'../../../../../../PhD Prep/Protege-5.5.0 for OSX/Ontologies/Ontologies for Protege/DQ_Metrics_Ontology.owl'

In [3]:
g = Graph()
g.parse (ontology_path,format='ttl')
gN = Namespace('http://www.semanticweb.org/janlucasdeinhard/ontologies/2023/2/untitled-ontology-9/')

df = pd.read_excel(
    io=path_to_dq_excel,
    sheet_name='DQ Checks'
)

In [4]:
person_sets = [
    'Data Owner',
    'Data Steward',
    'Data Quality Lead'
]

persons = [k for k in df[person_sets[0]].unique().tolist() if not k is np.nan]

item = persons[0]

In [5]:
item

'Sandra Freudl'

In [53]:
gN.PERSON

rdflib.term.URIRef('http://www.semanticweb.org/janlucasdeinhard/ontologies/2023/2/untitled-ontology-9/PERSON')

In [31]:
RDF.type

rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type')

In [54]:
'http://www.semanticweb.org/janlucasdeinhard/ontologies/2023/2/untitled-ontology-9/'+item.upper()

'http://www.semanticweb.org/janlucasdeinhard/ontologies/2023/2/untitled-ontology-9/'